In [2]:
%matplotlib widget
import os
import sys
sys.path.insert(0, '/data/visitors/cosaxs/sw/cosaxs-tools')
import h5py
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from ipywidgets import interact
from cormap import make_cormap
from multiprocessing import Pool
from scipy.signal import argrelmax


def load_data(fname):
    data = {}
    with h5py.File(fname, 'r') as fh:
        data['q'] = fh['q'][:]
        data['I'] = fh['I'][:]
        data['sigma'] = fh['sigma'][:]
    master = fname.replace('process/azint', 'raw')
    #master = master.replace('_eiger_integrated', '')
    master = master.replace('_eiger', '')
    with h5py.File(master, 'r') as fh:
        data['i_0'] = fh['/entry/measurement/i_0'][:]
        data['i_t'] = fh['/entry/measurement/i_t'][:]
        data['dt'] = fh['/entry/instrument/eiger/count_time'][()]
        data['sample'] = fh['/entry/sample/description'][()]
    return data

  
def average(data, dark_current, plot=False):
    norm = data['i_t'] / (data['i_0'] - dark_current)
    normed = data['I'] / norm.reshape(-1, 1) / data['dt']
    errors = data['sigma'] / norm.reshape(-1, 1) / data['dt']
    data['normed'] = normed
    
    cormap = make_cormap(normed) > 0.01
    # find longest sequence of good shots
    shot_count = []
    for i in range(len(cormap)):
        shot_count.append(np.count_nonzero(cormap[i, i:]))
    start = np.argmax(shot_count)
    print(f'start: {start} number of good shots: {shot_count[start]}')
    good_shots = start + np.where(cormap[start, start:] == 1)[0]
    I = np.mean(normed[good_shots], axis=0)
    errors = np.sqrt(np.sum(errors[good_shots]**2, axis=0)) / len(good_shots)
    if plot:
        fig, ax = plt.subplots(2, 2, figsize=(9, 6))
        fig.suptitle(r'$\bf{Scan}:$ %d | $\bf{Sample}:$ %s' %(scan, data['sample'].decode()))
        ax[0, 0].plot(data['q'], I)
        y1 = I - errors
        y2 = I + errors
        ax[0, 0].plot(data['q'], y1, color='grey', linestyle='--')
        ax[0, 0].plot(data['q'], y2, color='grey', linestyle='--')
        ax[0, 0].fill_between(data['q'], y1, y2, facecolor="gray", alpha=0.3)
        ax[0, 0].set_yscale('log')
        #ax[0, 0].set_xscale('log')
        ax[0, 0].set_xlabel('q [Å⁻¹]')
        ax[0, 0].set_ylabel('I')
        
        ax[0, 1].imshow(cormap, cmap='RdYlGn')
        ax[0, 1].axhline(start, color='white', linestyle='--')
        ax[0, 1].axvline(start, color='white', linestyle='--')

        ax[1, 0].plot(norm)
        ax[1, 0].set_ylabel(r'$I_t/I_0$')

        ax1 = ax[1, 1]
        ax1.plot(data['i_t'], color='tab:blue')
        ax1.set_ylabel(r'$I_t$', color='tab:blue')
        ax2 = plt.twinx()
        ax2.plot(data['i_0'], color='tab:orange')
        ax2.set_ylabel(r'$I_0$', color='tab:orange')
        plt.tight_layout()
        
    return data['q'], I, errors, cormap

dark_current = 3.77e-10
folder = '/data/visitors/cosaxs/20200740/2021111812/process/azint/'

## Cormap

In [4]:
scan = 4872
fname = os.path.join(folder, 'scan-%d_eiger.h5' %scan)
data = load_data(fname)
q, I, errors, cormap = average(data, dark_current, plot=True)
#output_file = os.path.join(folder.replace('azint', 'results'), f'{scan}.dat')
#np.savetxt(output_file, np.column_stack((q, I, errors)), header=data['sample'].decode())

start: 83 number of good shots: 514


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Buffer subtraction

In [16]:
buffer_scan, sample_scan = 4878, 4879
fname = os.path.join(folder.replace('azint', 'results'), f'{buffer_scan}.dat')
buffer = np.loadtxt(fname, unpack=True)
                    
fname = os.path.join(folder.replace('azint', 'results'), f'{sample_scan}.dat')
sample = np.loadtxt(fname, unpack=True)
plt.figure()
plt.title(sample_scan)
plt.plot(sample[0], sample[1], label='Sample')
plt.plot(buffer[0], buffer[1], label='Buffer')
q = sample[0]

a = np.sum(sample[1]*q**2)
b = np.sum(buffer[1]*q**2)

diff = sample[1] - buffer[1]
errors = np.sqrt(buffer[2]**2 + sample[2]**2)
y1 = diff - errors
y2 = diff + errors
plt.plot(q, diff, label='Difference')
plt.plot(q, y1, color='grey', linestyle='--')
plt.plot(q, y2, color='grey', linestyle='--')
plt.fill_between(q, y1, y2, facecolor="gray", alpha=0.3)
plt.xscale('log')
plt.yscale('log')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

##  Analysis of shots with Cormap 

In [11]:
plt.figure()
plt.imshow(cormap, cmap='RdYlGn')

shot_count = []
for i in range(len(cormap)):
    shot_count.append(np.count_nonzero(cormap[i, i:]))
plt.figure()
plt.plot(shot_count)
plt.xlabel('Shot index')
plt.title('Number of good shots')
#print(argrelmax(np.array(shot_count), order=5))

plt.figure()
for start in [0, 122, 300]:
    good_shots = start + np.where(cormap[start, start:] == 1)[0]
    print(len(good_shots))
    I = np.mean(data['normed'][good_shots], axis=0)
    plt.plot(data['q'], I, label=str(start))
#plt.xscale('log')
plt.yscale('log')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/tmp/ipykernel_568/180428736.py:13: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

3
468
303


## Plot detector images

In [7]:
master = fname.replace('process/azint', 'raw')
master = master.replace('_eiger', '')
fh = h5py.File(master, 'r')
images = fh['/entry/instrument/eiger/data']

@interact(i=(0, len(images)-1))
def show_frame(i):
    img = images[i]
    img[img == 2**32-1] = 0
    #img = img * (1 - mask)
    vmin, vmax = np.percentile(img, [1, 99.99])
    plt.figure()
    plt.imshow(img, vmin=vmin, vmax=vmax)
    plt.colorbar()

interactive(children=(IntSlider(value=325, description='i', max=650), Output()), _dom_classes=('widget-interac…

In [11]:
start = 142
good_shots = start + np.where(cormap[start, start:] == 1)[0]

c2 = []
for i in range(len(good_shots)):
    j = good_shots[i]
    for k in good_shots[:i]:
        l = len(normed[j])
        assert(l > 0)
        chi2 = ((normed[j] - normed[k])**2 / (sigma[j]**2 + sigma[k]**2 + 1.0e-20)).sum() / (l-1)
        c2.append(chi2)
c2 = np.array(c2)

plt.figure()
_ = plt.hist(c2, bins=np.linspace(0.6, 1.4, 100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …